### **Lymphoma**

In [ ]:
import tensorflow as tf
from tensorflow.keras.layers import Input, Conv2D, MaxPooling2D, GlobalAveragePooling2D, Concatenate, Dense, Dropout
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.applications import VGG19
from sklearn.model_selection import train_test_split
import numpy as np
import os
import matplotlib.pyplot as plt
from glob import glob
from PIL import Image

# Mount Google Drive
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

base_path = '/content/drive/My Drive/Lymphoma'  # Update with your dataset path
lymph_mcl_path = os.path.join(base_path, 'lymph_mcl')
lymph_fl_path = os.path.join(base_path, 'lymph_fl')
lymph_cll_path = os.path.join(base_path, 'lymph_cll')

# Print the contents of the folders to verify paths
print("Contents of lymph_mcl folder:")
print(os.listdir(lymph_mcl_path))

print("Contents of lymph_fl folder:")
print(os.listdir(lymph_fl_path))

print("Contents of lymph_cll folder:")
print(os.listdir(lymph_cll_path))

# Parameters
input_shape = (256, 256, 3)  # Resize images to 256x256
num_classes = 3

# Load and preprocess images
def load_images_from_folder(folder, label, size):
    images = []
    labels = []
    for ext in ('*.jpg', '*.jpeg', '*.png'):
        for filename in glob(os.path.join(folder, ext)):
            try:
                img = Image.open(filename).resize(size)
                img = np.array(img)
                if img is not None and len(img.shape) == 3 and img.shape[2] == 3:
                    images.append(img)
                    labels.append(label)
            except Exception as e:
                print(f"Error loading image {filename}: {e}")
    print(f'Loaded {len(images)} images from {folder}')
    return images, labels

lymph_mcl_images, lymph_mcl_labels = load_images_from_folder(lymph_mcl_path, 0, (256, 256))
lymph_fl_images, lymph_fl_labels = load_images_from_folder(lymph_fl_path, 1, (256, 256))
lymph_cll_images, lymph_cll_labels = load_images_from_folder(lymph_cll_path, 2, (256, 256))

# Combine and split the data
images = np.array(lymph_mcl_images + lymph_fl_images + lymph_cll_images)
labels = np.array(lymph_mcl_labels + lymph_fl_labels + lymph_cll_labels)

print(f'Total images: {len(images)}')
print(f'Total labels: {len(labels)}')

if len(images) == 0:
    raise ValueError("No images loaded. Please check the dataset paths and file extensions.")

x_train, x_test, y_train, y_test = train_test_split(images, labels, test_size=0.2, random_state=42)

# Normalize the data
x_train = x_train.astype('float32') / 255.0
x_test = x_test.astype('float32') / 255.0

# Convert labels to categorical
y_train = tf.keras.utils.to_categorical(y_train, num_classes)
y_test = tf.keras.utils.to_categorical(y_test, num_classes)

# Pyramid pooling module function
def build_ippm(input_tensor, pool_sizes):
    pyramid_pooling_outputs = []
    for pool_size in pool_sizes:
        x = tf.image.resize(input_tensor,
                            (input_tensor.shape[1] * pool_size, input_tensor.shape[2] * pool_size))
        x = MaxPooling2D(pool_size)(x)
        x = Conv2D(128, (1, 1), activation='relu')(x)
        pyramid_pooling_outputs.append(x)
    return Concatenate()(pyramid_pooling_outputs)

# Backbone function
def backbone(input_shape, num_classes, pool_sizes):
    inputs = Input(shape=input_shape)
    base_model = VGG19(include_top=False, input_tensor=inputs)

    x = build_ippm(base_model.output, pool_sizes)
    x = Conv2D(128, (3, 3), padding='same', activation='relu')(x)
    x = GlobalAveragePooling2D()(x)
    x = Dense(128, activation='relu')(x)
    x = Dropout(0.5)(x)
    outputs = Dense(num_classes, activation='softmax')(x)

    model = Model(inputs, outputs)
    return model

# Initialize the model
pool_sizes = [2, 3, 4, 6]
model = backbone(input_shape, num_classes, pool_sizes)

# Compile the model
learning_rate = 0.00001
model.compile(optimizer=Adam(learning_rate=learning_rate),
              loss='categorical_crossentropy',
              metrics=['accuracy'])

# Train the model
history = model.fit(x_train, y_train, epochs=10, batch_size=8, validation_data=(x_test, y_test))

# Evaluate the model
test_loss, test_accuracy = model.evaluate(x_test, y_test, verbose=2)
print(f'Test accuracy: {test_accuracy:.4f}')

# Plotting the results
plt.figure(figsize=(12, 4))
plt.subplot(1, 2, 1)
plt.plot(history.history['accuracy'])
plt.plot(history.history['val_accuracy'])
plt.title('Model accuracy')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.legend(['Train', 'Validation'], loc='upper left')

plt.subplot(1, 2, 2)
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('Model loss')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.legend(['Train', 'Validation'], loc='upper left')

plt.tight_layout()
plt.show()



### **Oral Cancer**

In [ ]:
import tensorflow as tf
from tensorflow.keras.layers import Input, Conv2D, MaxPooling2D, UpSampling2D, Concatenate, GlobalAveragePooling2D, Dense, BatchNormalization, Dropout
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.applications import MobileNetV2
from sklearn.model_selection import train_test_split
import numpy as np
import os
import matplotlib.pyplot as plt
from glob import glob
from PIL import Image

# Mount Google Drive
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

# Define paths
base_path = '/content/drive/My Drive/Oral Cancer'  # Update with your dataset path
oral_scc_path = os.path.join(base_path, 'oral_scc')
oral_normal_path = os.path.join(base_path, 'oral_normal')

# Print the contents of the folders to verify paths
print("Contents of oral_scc folder:")
print(os.listdir(oral_scc_path))

print("Contents of oral_normal folder:")
print(os.listdir(oral_normal_path))

# Parameters
input_shape = (256, 256, 3)  # Resize images to 256x256
num_classes = 2

# Load and preprocess images
def load_images_from_folder(folder, label, size):
    images = []
    labels = []
    for ext in ('*.jpg', '*.jpeg', '*.png'):
        for filename in glob(os.path.join(folder, ext)):
            try:
                img = Image.open(filename).resize(size)
                img = np.array(img)
                if img is not None and len(img.shape) == 3 and img.shape[2] == 3:
                    images.append(img)
                    labels.append(label)
            except Exception as e:
                print(f"Error loading image {filename}: {e}")
    print(f'Loaded {len(images)} images from {folder}')
    return images, labels

oral_scc_images, oral_scc_labels = load_images_from_folder(oral_scc_path, 0, (256, 256))
oral_normal_images, oral_normal_labels = load_images_from_folder(oral_normal_path, 1, (256, 256))

# Combine and split the data
images = np.array(oral_scc_images + oral_normal_images)
labels = np.array(oral_scc_labels + oral_normal_labels)

print(f'Total images: {len(images)}')
print(f'Total labels: {len(labels)}')

if len(images) == 0:
    raise ValueError("No images loaded. Please check the dataset paths and file extensions.")

x_train, x_test, y_train, y_test = train_test_split(images, labels, test_size=0.2, random_state=42)

# Normalize the data
x_train = x_train.astype('float32') / 255.0
x_test = x_test.astype('float32') / 255.0

# Convert labels to categorical
y_train = tf.keras.utils.to_categorical(y_train, num_classes)
y_test = tf.keras.utils.to_categorical(y_test, num_classes)

def create_mobilenetv2_model(input_shape, num_classes):
    base_model = MobileNetV2(weights='imagenet', include_top=False, input_shape=input_shape)
    base_model.trainable = False  # Freeze the base model

    inputs = Input(shape=input_shape)
    x = base_model(inputs, training=False)
    x = GlobalAveragePooling2D()(x)
    x = Dense(128, activation='relu')(x)
    x = Dropout(0.5)(x)
    outputs = Dense(num_classes, activation='softmax')(x)

    model = Model(inputs, outputs)
    return model

# Initialize the model
model = create_mobilenetv2_model(input_shape, num_classes)

# Compile the model
model.compile(optimizer=Adam(), loss='categorical_crossentropy', metrics=['accuracy'])

# Train the model
history = model.fit(x_train, y_train, epochs=25, batch_size=8, validation_data=(x_test, y_test))

# Evaluate the model
test_loss, test_accuracy = model.evaluate(x_test, y_test, verbose=2)
print(f'Test accuracy: {test_accuracy:.4f}')

# Plotting the results
plt.figure(figsize=(12, 4))
plt.subplot(1, 2, 1)
plt.plot(history.history['accuracy'])
plt.plot(history.history['val_accuracy'])
plt.title('Model accuracy')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.legend(['Train', 'Validation'], loc='upper left')

plt.subplot(1, 2, 2)
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('Model loss')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.legend(['Train', 'Validation'], loc='upper left')

plt.tight_layout()
plt.show()
